# OpenBugger
This notebook is a self-contained demo of the OpenBugger package that automatically bugs python code using LibCST

This plan for the notebook is to develop all the components necessary to construct a pipeline that
starting from a python script, it extracts its Concrete Syntax Tree and use it to apply a sequence of revertable syntactical code-mutation to automatically generate training data for debugging language models.

1. Tools to ensure that the sequence of modification is consistent and does not overwrite previously introduced mutations.
2. Bugger class to store the local context of multiple bugs that can be applied in a chain.
3. InverseTransformer class that is able to reverse the transformation of any other transformer sharing the same bugger context.
4. 5/12 example LibCST transformers that can each implement a different logical bug.  
5. Use the InverseTransformer to generate accurate debugging instructions to be used as training data for Large Language Models.

## LibCST

In [1]:
from libcst.codemod import CodemodContext, Codemod
from libcst.metadata import MetadataWrapper
from libcst import Equal, GreaterThanEqual, GreaterThan, CSTNode, Module
from typing import List
from copy import deepcopy
import libcst as cst
from libcst.codemod import CodemodContext, ContextAwareTransformer, ContextAwareVisitor
from libcst.metadata import BatchableMetadataProvider, PositionProvider, CodePosition, CodeRange
import libcst.matchers as m
import uuid
from openbugger.bugger import deep_equals_with_print

In [2]:
script = "while x < y[10]:\n\tprint(x)\n\tx = x[2] + 1+y[2:10]"
script2 = "while x < y[10]:\n\tprint(x)\n\tx = x[2] + 1+z[2:10]"

Module = cst.parse_module(script)
Module2 = cst.parse_module(script2)
deep_equals_with_print(Module, Module2)

Value mismatch in field 'value': y != z
Value mismatch in field 'value': Name(
    value='y',
    lpar=[],
    rpar=[],
) != Name(
    value='z',
    lpar=[],
    rpar=[],
)
Value mismatch in field 'right': Subscript(
    value=Name(
        value='y',
        lpar=[],
        rpar=[],
    ),
    slice=[
        SubscriptElement(
            slice=Slice(
                lower=Integer(
                    value='2',
                    lpar=[],
                    rpar=[],
                ),
                upper=Integer(
                    value='10',
                    lpar=[],
                    rpar=[],
                ),
                step=None,
                first_colon=Colon(
                    whitespace_before=SimpleWhitespace(
                        value='',
                    ),
                    whitespace_after=SimpleWhitespace(
                        value='',
                    ),
                ),
                second_colon=MaybeSentinel.DEFAULT,
      

False

Useful LibCstDocs: 

https://libcst.readthedocs.io/en/latest/metadata.html#position-metadata

https://libcst.readthedocs.io/en/latest/_modules/libcst/metadata/position_provider.html#PositionProvider

https://libcst.readthedocs.io/en/latest/_modules/libcst/metadata/position_provider.html#WhitespaceInclusivePositionProvidingCodegenState


https://libcst.readthedocs.io/en/latest/parser.html

## PositionContextUpdater and is_modified
Because we want to apply potentially random changes to the code we need to introduce some simple helper methods, contained in the is_modified function that use the meta-data saved from transformers after modifyng a node to prevent any other transformer from applying further modifications to the either Node, its parent or its childrens. 

Since each node modification might introduce or remove code we also use the PositionContextUpdater to maintain the context scratch consistent after each mutation.

In [3]:
from openbugger.context import is_modified, save_modified, is_parent_CodeRange, is_child_CodeRange, is_equal_Coderange, PositionContextUpdater

## Bugger and InverseTransformer

In [4]:
from openbugger.bugger import Bugger, TestTransformer, bugger_example, InverseTransformer

### Example Debugging Output Using the Test Tranformer

In [5]:
transformers = [TestTransformer]
# Get the script as a string it should have  while loop and take multiple lines
script = "while x < y[10]:\n\tprint(x)\n\tx = x[2] + 1+y[2:10]"
bugger_example(transformers,script)


original_code
while x < y[10]:
	print(x)
	x = x[2] + 1+y[2:10]
tainted_code
while x < y[10]:
	print(x)
	x = x[2] + 1+y[2:10]
The result of deep_equals between the concrete syntax tree of the original and the bugged code is True
Checking for bugs...
The following Node has a bug of type TestTransformer-b87c starting at line 1, column 12 and ending at line 1, column 14.
The bug can be fixed by substituting the bugged code-string <10> with the following code-string <10>
The following Node has a bug of type TestTransformer-b87c starting at line 3, column 7 and ending at line 3, column 8.
The bug can be fixed by substituting the bugged code-string <2> with the following code-string <2>
The following Node has a bug of type TestTransformer-b87c starting at line 3, column 16 and ending at line 3, column 20.
The bug can be fixed by substituting the bugged code-string <2:10> with the following code-string <2:10>
Debugging...
clean_code
while x < y[10]:
	print(x)
	x = x[2] + 1+y[2:10]
Checking if 

# Example Bugs 

In this section of the notebook we develop the 5/12 example bugs using LibCST ContextAwareTransformer that use the is_modified method to check if the node was already targeted by a mutation and then apply the mutation and save the mutation to the scratchpad using save_modified method.

The bugs are:

1. incorrect_comparison_operator - Done
2. comparison_swap - Done
3. forgetting_to_update_variable - Done
4. infinite_loop - Done
5. off_by_k_index - Done
6. incorrect_return_value
7. incorrect_boolean_operator
8. using_wrong_type_of_loop
9. using_loop_variable_outside_loop
10. using_variable_before_assignment
11. using_wrong_variable_scope
12. incorrect_use_of_exception_handling
13. incorrect_function_call

The current approach only allows for the scratchpad context to be passed as initialization to the transformers, therefore we use generator functions for the bugs that could take multiple inputs like which operator to swap. These transformers apply the bug to ALL the target instances they find, we will derive some wrapper in the later part of the notebook to control the number of bugs or target a specific code-range.

## LogicalBugs

In [6]:
from openbugger.bugs.logical import gen_ComparisonTargetTransfomer, ComparisonSwapTransformer

### Incorrect Comparison Operator
This bug takes as input two libcst comparison operators and swaps every instance of the first for the second

In [7]:
transformers = [gen_ComparisonTargetTransfomer('==','!=')]
# Get the script as a string
script = "x == 1 + 2 == 3 + 2 != 3 + 4 > 3"

bugger_example(transformers,script)

original_code
x == 1 + 2 == 3 + 2 != 3 + 4 > 3
tainted_code
x != 1 + 2 != 3 + 2 != 3 + 4 > 3
The result of deep_equals between the concrete syntax tree of the original and the bugged code is False
Checking for bugs...
The following Node has a bug of type ComparisonTargetTransformer-a2c2 starting at line 1, column 1 and ending at line 1, column 10.
The bug can be fixed by substituting the bugged code-string < != 1 + 2> with the following code-string < == 1 + 2>
The following Node has a bug of type ComparisonTargetTransformer-a2c2 starting at line 1, column 10 and ending at line 1, column 19.
The bug can be fixed by substituting the bugged code-string < != 3 + 2> with the following code-string < == 3 + 2>
Debugging...
clean_code
x == 1 + 2 == 3 + 2 != 3 + 4 > 3
Checking if the debugged code is equal to the original code..
The result of deep_equals between the concrete syntax tree of the original and debugged code is True


### Comparison Swap

In [8]:
transformers = [ComparisonSwapTransformer]
# Get the script as a string
script = "x == 1 + 2 == 3 + 2 != 3 + 4 > 3 "
bugger_example(transformers,script)

original_code
x == 1 + 2 == 3 + 2 != 3 + 4 > 3 
tainted_code
1 + 2 == x == 3 + 2 != 3 + 4 > 3 
The result of deep_equals between the concrete syntax tree of the original and the bugged code is False
Checking for bugs...
The following Node has a bug of type ComparisonSwapTransformer-26b2 starting at line 1, column 0 and ending at line 1, column 32.
The bug can be fixed by substituting the bugged code-string <1 + 2 == x == 3 + 2 != 3 + 4 > 3> with the following code-string <x == 1 + 2 == 3 + 2 != 3 + 4 > 3>
Debugging...
clean_code
x == 1 + 2 == 3 + 2 != 3 + 4 > 3 
Checking if the debugged code is equal to the original code..
The result of deep_equals between the concrete syntax tree of the original and debugged code is True


## ControlFLow Bugs

In [9]:
from openbugger.bugs.controlflow import ForgettingToUpdateVariableTransformer, InfiniteWhileTransformer,gen_OffByKIndexTransformer

### ForgettingToUpdateVariable

In [10]:
transformers = [ForgettingToUpdateVariableTransformer]
# Get the script as a string
script = "while x == 1 + 2 == 3 + 2 != 3 + 4 > 3 : \n  y = 1 + 2"
bugger_example(transformers,script)

original_code
while x == 1 + 2 == 3 + 2 != 3 + 4 > 3 : 
  y = 1 + 2
tainted_code
while x == 1 + 2 == 3 + 2 != 3 + 4 > 3 : 
  y = y
The result of deep_equals between the concrete syntax tree of the original and the bugged code is False
Checking for bugs...
The following Node has a bug of type ForgettingToUpdateVariableTransformer-9c62 starting at line 2, column 2 and ending at line 2, column 7.
The bug can be fixed by substituting the bugged code-string <y = y> with the following code-string <y = 1 + 2>
Debugging...
clean_code
while x == 1 + 2 == 3 + 2 != 3 + 4 > 3 : 
  y = 1 + 2
Checking if the debugged code is equal to the original code..
The result of deep_equals between the concrete syntax tree of the original and debugged code is True


### Infinite While Loop

In [11]:
transformers = [InfiniteWhileTransformer]
# Get the script as a string
script = "while x == 1 + 2 == 3 + 2 != 3 + 4 > 3: \n  y = 1 + 2"
bugger_example(transformers,script)

original_code
while x == 1 + 2 == 3 + 2 != 3 + 4 > 3: 
  y = 1 + 2
tainted_code
while True: 
  y = 1 + 2
The result of deep_equals between the concrete syntax tree of the original and the bugged code is False
Checking for bugs...
The following Node has a bug of type InfiniteWhileTransformer-e885 starting at line 1, column 0 and ending at line 2, column 11.
The bug can be fixed by substituting the bugged code-string <while True: 
  y = 1 + 2
> with the following code-string <while x == 1 + 2 == 3 + 2 != 3 + 4 > 3: 
  y = 1 + 2
>
Debugging...
clean_code
while x == 1 + 2 == 3 + 2 != 3 + 4 > 3: 
  y = 1 + 2
Checking if the debugged code is equal to the original code..
The result of deep_equals between the concrete syntax tree of the original and debugged code is True


### OffByKIndex 

In [12]:
transformers = [gen_OffByKIndexTransformer(1)]
# Get the script as a string
script = "while x == 1 + 2 == 3 + 2 != 3 + 4 > 3: \n  y = 1 + 2\nx[1:2]\nx[1]"
bugger_example(transformers,script)

original_code
while x == 1 + 2 == 3 + 2 != 3 + 4 > 3: 
  y = 1 + 2
x[1:2]
x[1]
tainted_code
while x == 1 + 2 == 3 + 2 != 3 + 4 > 3: 
  y = 1 + 2
x[2:3]
x[2]
The result of deep_equals between the concrete syntax tree of the original and the bugged code is False
Checking for bugs...
The following Node has a bug of type OffByKIndexTransformer-d1a2 starting at line 3, column 2 and ending at line 3, column 5.
The bug can be fixed by substituting the bugged code-string <2:3> with the following code-string <1:2>
The following Node has a bug of type OffByKIndexTransformer-d1a2 starting at line 4, column 2 and ending at line 4, column 3.
The bug can be fixed by substituting the bugged code-string <2> with the following code-string <1>
Debugging...
clean_code
while x == 1 + 2 == 3 + 2 != 3 + 4 > 3: 
  y = 1 + 2
x[1:2]
x[1]
Checking if the debugged code is equal to the original code..
The result of deep_equals between the concrete syntax tree of the original and debugged code is True


### Incorrect Exception Handling
Catching the wrong exceptions, or incorrect use of the `try/except/finally` block.

In [13]:
from openbugger.bugs.controlflow import IncorrectExceptionHandlerTransformer

In [14]:
script= """try:
    x = 1 / 0
except ZeroDivisionError:
    print('Zero division error!')
"""
expected_output= """try:
    x = 1 / 0
except:
    print('Zero division error!')
"""

In [15]:
transformers = [IncorrectExceptionHandlerTransformer]
bugger_example(transformers,script)

original_code
try:
    x = 1 / 0
except ZeroDivisionError:
    print('Zero division error!')

tainted_code
try:
    x = 1 / 0
except :
    print('Zero division error!')

The result of deep_equals between the concrete syntax tree of the original and the bugged code is False
Checking for bugs...
The following Node has a bug of type IncorrectExceptionHandlerTransformer-87fb starting at line 3, column 0 and ending at line 4, column 33.
The bug can be fixed by substituting the bugged code-string <except :
    print('Zero division error!')
> with the following code-string <except ZeroDivisionError:
    print('Zero division error!')
>
Debugging...
clean_code
try:
    x = 1 / 0
except ZeroDivisionError:
    print('Zero division error!')

Checking if the debugged code is equal to the original code..
The result of deep_equals between the concrete syntax tree of the original and debugged code is True


### Missing Argument Transformer
Calling functions in the wrong sequence.

In [16]:
from openbugger.bugs.controlflow import MissingArgumentTransformer

In [17]:
script= """print(str(123))"""
expected_output = """print(str())"""
transformers = [MissingArgumentTransformer]
bugger_example(transformers,script)


original_code
print(str(123))
tainted_code
print(str())
The result of deep_equals between the concrete syntax tree of the original and the bugged code is False
Checking for bugs...
The following Node has a bug of type MissingArgumentTransformer-14d8 starting at line 1, column 6 and ending at line 1, column 11.
The bug can be fixed by substituting the bugged code-string <str()> with the following code-string <str(123)>
Debugging...
clean_code
print(str(123))
Checking if the debugged code is equal to the original code..
The result of deep_equals between the concrete syntax tree of the original and debugged code is True


### ReturningEarly
Returning from a function before all the necessary computations have been made.

In [18]:
from openbugger.bugs.controlflow import ReturningEarlyTransformer



In [19]:
script= """def add(x, y):
    return x + y
    print('This will not be printed.')
"""
expected_output = """def add(x, y):
    return
    print('This will not be printed.')
"""
transformers = [ReturningEarlyTransformer]
bugger_example(transformers,script)

original_code
def add(x, y):
    return x + y
    print('This will not be printed.')

tainted_code
def add(x, y):
    return
    print('This will not be printed.')

The result of deep_equals between the concrete syntax tree of the original and the bugged code is False
Checking for bugs...
The following Node has a bug of type ReturningEarlyTransformer-9b55 starting at line 2, column 4 and ending at line 2, column 10.
The bug can be fixed by substituting the bugged code-string <return> with the following code-string <return x + y>
Debugging...
clean_code
def add(x, y):
    return x + y
    print('This will not be printed.')

Checking if the debugged code is equal to the original code..
The result of deep_equals between the concrete syntax tree of the original and debugged code is True


## Data Related Bugs

### Incorrect variable initialization
Variables are initialized with incorrect values.

In [20]:
from openbugger.bugs.data import IncorrectVariableInitializationTransformer

In [21]:
script = """var1 = 5
var2 = "hello"
var3 = [1, 2, 3]
var4 = {"key": "value"}
var5 = True
var6 = None
"""
possible_output = """var1 = 2
var2 = "bar"
var3 = ['a', 'b', 'c']
var4 = {}
var5 = False
var6 = None
"""
transformers = [IncorrectVariableInitializationTransformer]
bugger_example(transformers,script)

original_code
var1 = 5
var2 = "hello"
var3 = [1, 2, 3]
var4 = {"key": "value"}
var5 = True
var6 = None

tainted_code
var1 = 5
var2 = "bar"
var3 = [3, 2, 3]
var4 = {"key": "baz"}
var5 = False
var6 = 0

The result of deep_equals between the concrete syntax tree of the original and the bugged code is False
Checking for bugs...
The following Node has a bug of type IncorrectVariableInitializationTransformer-a070 starting at line 1, column 0 and ending at line 1, column 8.
The bug can be fixed by substituting the bugged code-string <var1 = 5> with the following code-string <var1 = 5>
The following Node has a bug of type IncorrectVariableInitializationTransformer-a070 starting at line 2, column 0 and ending at line 2, column 12.
The bug can be fixed by substituting the bugged code-string <var2 = "bar"> with the following code-string <var2 = "hello">
The following Node has a bug of type IncorrectVariableInitializationTransformer-a070 starting at line 3, column 7 and ending at line 3, column 16

### Variable Name Typo
Mistyping the names of variables, causing them to be treated as new, uninitialized variables.

In [22]:
from openbugger.bugs.data import VariableNameTypoTransformer

In [23]:
script = """var1 = 5
var2 = "hello"
var1 = var2
"""
possible_output = """var1 = 5
var2 = "hello"
var1_var2 = var2
"""
transformers = [VariableNameTypoTransformer]
bugger_example(transformers,script)

original_code
var1 = 5
var2 = "hello"
var1 = var2

tainted_code
var1var1 = 5
var2var2 = "hello"
var1var1 = var2

The result of deep_equals between the concrete syntax tree of the original and the bugged code is False
Checking for bugs...
The following Node has a bug of type VariableNameTypoTransformer-45bb starting at line 1, column 0 and ending at line 1, column 12.
The bug can be fixed by substituting the bugged code-string <var1var1 = 5> with the following code-string <var1 = 5>
The following Node has a bug of type VariableNameTypoTransformer-45bb starting at line 2, column 0 and ending at line 2, column 18.
The bug can be fixed by substituting the bugged code-string <var2var2 = "hello"> with the following code-string <var2 = "hello">
The following Node has a bug of type VariableNameTypoTransformer-45bb starting at line 3, column 0 and ending at line 3, column 15.
The bug can be fixed by substituting the bugged code-string <var1var1 = var2> with the following code-string <var1 = var

### Mutable Default Arguments
Using mutable types (like lists or dictionaries) as default function arguments.

In [24]:
from openbugger.bugs.data import MutableDefaultArgumentTransformer

In [25]:
script = """def func1(arg1, arg2, arg3):
    return arg1 + arg2"""

transformers = [MutableDefaultArgumentTransformer]
bugger_example(transformers,script)

original_code
def func1(arg1, arg2, arg3):
    return arg1 + arg2
tainted_code
def func1(arg1, arg2, arg3 = []):
    arg3.append(1)
    return arg1 + arg2
The result of deep_equals between the concrete syntax tree of the original and the bugged code is False
Checking for bugs...
The following Node has a bug of type MutableDefaultArgumentTransformer-1687 starting at line 1, column 0 and ending at line 3, column 22.
The bug can be fixed by substituting the bugged code-string <def func1(arg1, arg2, arg3 = []):
    arg3.append(1)
    return arg1 + arg2
> with the following code-string <def func1(arg1, arg2, arg3):
    return arg1 + arg2
>
Debugging...
clean_code
def func1(arg1, arg2, arg3):
    return arg1 + arg2
Checking if the debugged code is equal to the original code..
The result of deep_equals between the concrete syntax tree of the original and debugged code is True


### Using Variable before Assignment

In [26]:
from openbugger.bugs.data import UseBeforeDefinitionTransformer

In [27]:
script = """def add_two_numbers(a, b):
    return a + b"""
expected_output = """def add_two_numbers(a, b):
    return a + temp  # 'temp' is used before it's defined
    temp = b"""
transformers = [UseBeforeDefinitionTransformer]
bugger_example(transformers,script)

original_code
def add_two_numbers(a, b):
    return a + b
tainted_code
def add_two_numbers(a, b):
    ba = b + 1
    return a + b
The result of deep_equals between the concrete syntax tree of the original and the bugged code is False
Checking for bugs...
The following Node has a bug of type UseBeforeDefinitionTransformer-c4e1 starting at line 1, column 0 and ending at line 3, column 16.
The bug can be fixed by substituting the bugged code-string <def add_two_numbers(a, b):
    ba = b + 1
    return a + b
> with the following code-string <def add_two_numbers(a, b):
    return a + b
>
Debugging...
clean_code
def add_two_numbers(a, b):
    return a + b
Checking if the debugged code is equal to the original code..
The result of deep_equals between the concrete syntax tree of the original and debugged code is True


## Type-related Bugs

# Incorrect type used: 
Using an integer when a string was expected, or vice versa.

In [28]:
from openbugger.bugs.type import IncorrectTypeTransformer

In [29]:
script = """def function():
    return 10"""
# script  = """def function():
#     return "10" """
# # script = """def function():
# #     return 10.5"""
transformers = [IncorrectTypeTransformer]
bugger_example(transformers,script)


original_code
def function():
    return 10
tainted_code
def function():
    return "10"
The result of deep_equals between the concrete syntax tree of the original and the bugged code is False
Checking for bugs...
The following Node has a bug of type IncorrectTypeTransformer-694c starting at line 2, column 11 and ending at line 2, column 15.
The bug can be fixed by substituting the bugged code-string <"10"> with the following code-string <10>
Debugging...
clean_code
def function():
    return 10
Checking if the debugged code is equal to the original code..
The result of deep_equals between the concrete syntax tree of the original and debugged code is True


### Calling non existing Methods

In [30]:

from openbugger.bugs.type import NonExistingMethodTransformer

In [31]:
script = """my_list = [1, 2, 3]
my_list.append(4)
"""
transformers = [NonExistingMethodTransformer]
bugger_example(transformers,script)

original_code
my_list = [1, 2, 3]
my_list.append(4)

tainted_code
my_list = [1, 2, 3]
my_list.update(4)

The result of deep_equals between the concrete syntax tree of the original and the bugged code is False
Checking for bugs...
The following Node has a bug of type NonExistingMethodTransformer-7ff6 starting at line 2, column 0 and ending at line 2, column 17.
The bug can be fixed by substituting the bugged code-string <my_list.update(4)> with the following code-string <my_list.append(4)>
Debugging...
clean_code
my_list = [1, 2, 3]
my_list.append(4)

Checking if the debugged code is equal to the original code..
The result of deep_equals between the concrete syntax tree of the original and debugged code is True


# Using wrong type of loop
 Using a `for` loop when a `while` loop would be more appropriate, and vice versa.

In [32]:


from openbugger.bugs.type import SwapForTransformer

In [33]:
script = """for i in range(10):
    print(i)"""

transformers = [SwapForTransformer]
bugger_example(transformers,script)

original_code
for i in range(10):
    print(i)
tainted_code
while next(range(10)):
    print(i)
The result of deep_equals between the concrete syntax tree of the original and the bugged code is False
Checking for bugs...
The following Node has a bug of type SwapForTransformer-327c starting at line 1, column 0 and ending at line 2, column 12.
The bug can be fixed by substituting the bugged code-string <while next(range(10)):
    print(i)
> with the following code-string <for i in range(10):
    print(i)
>
Debugging...
clean_code
for i in range(10):
    print(i)
Checking if the debugged code is equal to the original code..
The result of deep_equals between the concrete syntax tree of the original and debugged code is True


## Numpy Bugs

### numpy creation bug
Initalizes a numpy array with empty instead of array

In [34]:
from openbugger.bugs.numpy import NumpyArrayCreationTransformer

In [35]:
script = """def create_array():
    arr = numpy.array([1, 2, 3])
    return arr"""
transformers = [NumpyArrayCreationTransformer]
bugger_example(transformers,script)

original_code
def create_array():
    arr = numpy.array([1, 2, 3])
    return arr
tainted_code
def create_array():
    arr = numpy.empty([1, 2, 3])
    return arr
The result of deep_equals between the concrete syntax tree of the original and the bugged code is False
Checking for bugs...
The following Node has a bug of type NumpyArrayCreationTransformer-4d97 starting at line 2, column 10 and ending at line 2, column 32.
The bug can be fixed by substituting the bugged code-string <numpy.empty([1, 2, 3])> with the following code-string <numpy.array([1, 2, 3])>
Debugging...
clean_code
def create_array():
    arr = numpy.array([1, 2, 3])
    return arr
Checking if the debugged code is equal to the original code..
The result of deep_equals between the concrete syntax tree of the original and debugged code is True


### Numpy Sort to argsort 

In [36]:
from openbugger.bugs.numpy import NumpyMethodMisuseTransformer

In [37]:
script = """
import numpy as np

def sort_array():
    arr = np.array([3, 1, 2])
    sorted_arr = np.sort(arr)
    return sorted_arr
"""
transformers = [NumpyMethodMisuseTransformer]
bugger_example(transformers,script)

original_code

import numpy as np

def sort_array():
    arr = np.array([3, 1, 2])
    sorted_arr = np.sort(arr)
    return sorted_arr

tainted_code

import numpy as np

def sort_array():
    arr = np.array([3, 1, 2])
    sorted_arr = np.argsort(arr)
    return sorted_arr

The result of deep_equals between the concrete syntax tree of the original and the bugged code is False
Checking for bugs...
The following Node has a bug of type NumpyMethodMisuseTransformer-f5f3 starting at line 6, column 17 and ending at line 6, column 32.
The bug can be fixed by substituting the bugged code-string <np.argsort(arr)> with the following code-string <np.sort(arr)>
Debugging...
clean_code

import numpy as np

def sort_array():
    arr = np.array([3, 1, 2])
    sorted_arr = np.sort(arr)
    return sorted_arr

Checking if the debugged code is equal to the original code..
The result of deep_equals between the concrete syntax tree of the original and debugged code is True


### WrongReshape

In [38]:

from openbugger.bugs.numpy import NumpyReshapeMisuseTransformer


In [39]:
script = """def reshape_array():
    arr = np.array([[1, 2, 3], [4, 5, 6]])
    reshaped_arr = arr.reshape([2, 3])
    return reshaped_arr"""
transformers = [NumpyReshapeMisuseTransformer]
bugger_example(transformers,script)

original_code
def reshape_array():
    arr = np.array([[1, 2, 3], [4, 5, 6]])
    reshaped_arr = arr.reshape([2, 3])
    return reshaped_arr
tainted_code
def reshape_array():
    arr = np.array([[1, 2, 3], [4, 5, 6]])
    reshaped_arr = arr.reshape([3, 3])
    return reshaped_arr
The result of deep_equals between the concrete syntax tree of the original and the bugged code is False
Checking for bugs...
The following Node has a bug of type NumpyReshapeMisuseTransformer-5b78 starting at line 3, column 19 and ending at line 3, column 38.
The bug can be fixed by substituting the bugged code-string <arr.reshape([3, 3])> with the following code-string <arr.reshape([2, 3])>
Debugging...
clean_code
def reshape_array():
    arr = np.array([[1, 2, 3], [4, 5, 6]])
    reshaped_arr = arr.reshape([2, 3])
    return reshaped_arr
Checking if the debugged code is equal to the original code..
The result of deep_equals between the concrete syntax tree of the original and debugged code is True


### numpy arange

In [40]:
from openbugger.bugs.numpy import NumpyArangeMisuseTransformer


In [41]:
script= """data = np.arange(10)"""
transformers = [NumpyArangeMisuseTransformer]
bugger_example(transformers,script)

original_code
data = np.arange(10)
tainted_code
data = np.arange(10.4)
The result of deep_equals between the concrete syntax tree of the original and the bugged code is False
Checking for bugs...
The following Node has a bug of type NumpyArangeMisuseTransformer-618d starting at line 1, column 7 and ending at line 1, column 22.
The bug can be fixed by substituting the bugged code-string <np.arange(10.4)> with the following code-string <np.arange(10)>
Debugging...
clean_code
data = np.arange(10)
Checking if the debugged code is equal to the original code..
The result of deep_equals between the concrete syntax tree of the original and debugged code is True


### Numpy axis misuses

In [42]:
from openbugger.bugs.numpy import NumpyAxisMisuseTransformer


In [43]:
script = """data = numpy.array([[1, 2, 3], [4, 5, 6]])
sum_along_axis = numpy.sum(data, axis=16)"""
transformers = [NumpyAxisMisuseTransformer]
bugger_example(transformers,script)

original_code
data = numpy.array([[1, 2, 3], [4, 5, 6]])
sum_along_axis = numpy.sum(data, axis=16)
tainted_code
data = numpy.array([[1, 2, 3], [4, 5, 6]])
sum_along_axis = numpy.sum(data, axis = 1)
The result of deep_equals between the concrete syntax tree of the original and the bugged code is False
Checking for bugs...
The following Node has a bug of type NumpyAxisMisuseTransformer-d2d6 starting at line 2, column 17 and ending at line 2, column 42.
The bug can be fixed by substituting the bugged code-string <numpy.sum(data, axis = 1)> with the following code-string <numpy.sum(data, axis=16)>
Debugging...
clean_code
data = numpy.array([[1, 2, 3], [4, 5, 6]])
sum_along_axis = numpy.sum(data, axis=16)
Checking if the debugged code is equal to the original code..
The result of deep_equals between the concrete syntax tree of the original and debugged code is True


## Chaining Multiple Bugs

Bugs can be chained in a sequence and they are executed in an order, bugs do not modify nodes that have already been modified or that either their children or parent has been modified. As an example we are going to all InfiniteWhiletransformer, OffByKIndexTransformer and ComparisonTargetTransfomer over the script:

while x == 1 + 2 == 3 + 2 != 3 + 4 > z[4]:  y = 1 + 2\nx[1:2]\nx[1]\nx[1]==3 

Comparison target and OffByKIndex only modify targets that are outside the while target statement.

In [44]:
transformers = [InfiniteWhileTransformer,gen_OffByKIndexTransformer(1),gen_ComparisonTargetTransfomer("==",">=")]
# Get the script as a string
script = "while x == 1 + 2 == 3 + 2 != 3 + 4 > 3: \n  y = 1 + 2\nx[1:2]\nx[1]\nx[1]==3"
bugger_example(transformers,script)

original_code
while x == 1 + 2 == 3 + 2 != 3 + 4 > 3: 
  y = 1 + 2
x[1:2]
x[1]
x[1]==3
tainted_code
while True: 
  y = 1 + 2
x[2:3]
x[2]
x[2] >= 3
The result of deep_equals between the concrete syntax tree of the original and the bugged code is False
Checking for bugs...
The following Node has a bug of type InfiniteWhileTransformer-21dd starting at line 1, column 0 and ending at line 2, column 11.
The bug can be fixed by substituting the bugged code-string <while True: 
  y = 1 + 2
> with the following code-string <while x == 1 + 2 == 3 + 2 != 3 + 4 > 3: 
  y = 1 + 2
>
The following Node has a bug of type OffByKIndexTransformer-1750 starting at line 3, column 2 and ending at line 3, column 5.
The bug can be fixed by substituting the bugged code-string <2:3> with the following code-string <1:2>
The following Node has a bug of type OffByKIndexTransformer-1750 starting at line 4, column 2 and ending at line 4, column 3.
The bug can be fixed by substituting the bugged code-string <2> with 